In [1]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd

path= 'images'
languages = ['c', 'asm1', 'asm2']
filters = ['hsl', 'blur', 'merge']
command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

hhssll = [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)]
alpha = [0.0, 0.14674845465465456489765465478976546548]
manager = multiprocessing.Manager()
dataset = manager.list([])

def worker(filt, lang):
    output = []
    rpath = os.path.join(path, filt)

    for img in os.listdir(rpath):
        rimg = os.path.join(rpath, img)

        if os.path.isfile(rimg):
            
            if filt == 'hsl':
                for (h, s, l) in hhssll:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            dst = './current.bmp',
                            h = h,
                            s = s,
                            l = l
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'delta': delta,
                        'h': h,
                        's': s,
                        'l': l
                    })
            elif filt == 'blur':
                comm = command[filt].format(
                        language = lang,
                        filter = filt,
                        src = rimg,
                        dst = './current.bmp'
                    )

                delta = subprocess.check_output(comm, shell=True)
                delta = delta.strip()

                output.append({
                    'filename': img,
                    'language': lang,
                    'filter': filt,
                    'delta': delta,
                })
            elif filt == 'merge':
                for a in alpha:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            src2 = rimg,
                            dst = './current.bmp',
                            alpha = a
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'delta': delta,
                        'alpha': a
                    })
                    
    dataset.append(output)
                    
jobs = []
for filt, lang in itertools.product(filters, languages):
    p = multiprocessing.Process(target=worker, args=(filt, lang))
    jobs.append(p)
    p.start()

for job in jobs:
    job.join()

In [3]:
data = pd.DataFrame(dataset)

PandasError: DataFrame constructor not properly called!

In [5]:
data = []

for lst in dataset:
    data += lst

In [7]:
dst = pd.DataFrame(data)

In [10]:
dst.set_index(['filter', 'language'])
dst.to_csv('output.csv')